Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Custom Vision Service Model
* Create Workspace
* Register Demo model
* Build Image 
* Create Hub and Prepare for device and deploy
* Deploy Model

In [2]:
# For prod
!pip install azureml-core
!pip install azureml-contrib-iot --user
!pip install azure-mgmt-containerregistry   

  Using cached https://files.pythonhosted.org/packages/12/0b/7c2abac336dada5905b39835c98e0b02a30002b3c5941d68bbdb7762db9c/azureml_contrib_iot-1.0.17-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/54/8e/4e57078e34ce48d089c5e427327ebdb7a5002d807140a90948c3635bfc2e/azureml_core-1.0.17.1-py2.py3-none-any.whl


azureml-widgets 1.0.10 has requirement azureml-core==1.0.10.*, but you'll have azureml-core 1.0.17.1 which is incompatible.
azureml-train-core 1.0.10 has requirement azureml-core==1.0.10.*, but you'll have azureml-core 1.0.17.1 which is incompatible.
azureml-telemetry 1.0.10 has requirement azureml-core==1.0.10.*, but you'll have azureml-core 1.0.17.1 which is incompatible.
azureml-sdk 1.0.10 has requirement azureml-core==1.0.10.*, but you'll have azureml-core 1.0.17.1 which is incompatible.
azureml-pipeline-core 1.0.10 has requirement azureml-core==1.0.10.*, but you'll have azureml-core 1.0.17.1 which is incompatible.
azureml-contrib-tensorboard 1.0.6 has requirement azureml-core==1.0.6.*, but you'll have azureml-core 1.0.17.1 which is incompatible.
azureml-contrib-services 1.0.6 has requirement azureml-core==1.0.6.*, but you'll have azureml-core 1.0.17.1 which is incompatible.
azureml-contrib-server 1.0.6 has requirement azureml-core==1.0.6.*, but you'll have azureml-core 1.0.17.1 wh

In [3]:
import os
print(os.__file__)

C:\ProgramData\Miniconda3\envs\iotedge\lib\os.py


In [4]:
# Check core SDK version number
import azureml.core as azcore

print("SDK version:", azcore.VERSION)

SDK version: 1.0.17


### Create a Workspace
#### Change this cell from markdown to code and run this if you need to create a workspace 
#### Update the values for your workspace below
ws=Workspace.create(subscription_id="replace-with-subscription-id",
                resource_group="your-resource-group",
                name="your-workspace-name",
                location="eastus2")
                
ws.write_config()

In [5]:
#Initialize Workspace 
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Found the config file in: C:\Users\tedway\git\vision-ai-developer-kit\machine-learning-reference\notebooks\aml_config\config.json
aml-vdaik-1
tedway-vienna1-rg
eastus2
75f78a03-482f-4fd8-8c71-5ddc08f92726


### Register model
Download the model files from customvision.ai.

In [6]:
from azureml.core.model import Model

model = Model.register(model_path = "models/squeezenet-forks/orig/",
                      model_name = "forks_scissors",
                      tags = {"data": "customvision", "model": "object_detection", "type": "squeezenet"},
                      description = "Retrained forks and scissors based on Squeezenet",
                      workspace = ws)
print(model.name, model.url, model.version, model.id, model.created_time)

Registering model forks_scissors
forks_scissors aml://asset/4c875ec7dd4644bba74339b80608d005 3 forks_scissors:3 2019-03-07 20:08:34.817776+00:00


In [ ]:
from azureml.contrib.iot.model_converters import SnpeConverter

# submit a compile request
compile_request = SnpeConverter.convert_caffe_model(
    ws,
    source_model=model,
    mirror_content = True)
print(compile_request._operation_id)

In [ ]:
# wait for the request to complete
compile_request.wait_for_completion(show_output=True)

In [ ]:
# Get converted model
converted_model = compile_request.result
print(converted_model.name, converted_model.url, converted_model.version, converted_model.id, converted_model.created_time)

In [ ]:
from azureml.core.image import Image
from azureml.contrib.iot import IotContainerImage
print ('We will create an image for you now ...')
image_config = IotContainerImage.image_configuration(
                                 architecture="arm32v7",
                                 execution_script="main.py", 
                                 dependencies=["camera.py","iot.py","ipcprovider.py","utility.py", "frame_iterators.py"],
                                 docker_file="Dockerfile", 
                                 tags = ["squeezenet"],
                                 description = "Squeezenet model from customvision.ai")
image = Image.create(name = "forksscissors",
                     # this is the model object 
                     models = [converted_model],
                     image_config = image_config, 
                     workspace = ws)
image.wait_for_creation(show_output = True)

In [ ]:
# Parameter list

# Pick a name for what you want to call the module you deploy to the camera
module_name = "visionsample"

# Resource group in Azure 
resource_group_name= ws.resource_group
iot_rg="vaidk_"+resource_group_name

# Azure region where your services will be provisioned
iot_location="eastus2"

# Azure IoT Hub name
iot_hub_name="iothub-"+ ws.get_details()["name"]

# Pick a name for your camera
iot_device_id="vadik_"+ ws.get_details()["name"]

# Pick a name for the deployment configuration
iot_deployment_id="demovaidk"

In [ ]:
# Getting your container details
container_reg = ws.get_details()["containerRegistry"]
reg_name=container_reg.split("/")[-1]
container_url = "\"" + image.image_location + "\","
subscription_id = ws.subscription_id
print('{}'.format(image.image_location))
print('{}'.format(reg_name))
print('{}'.format(subscription_id))
from azure.mgmt.containerregistry import ContainerRegistryManagementClient
from azure.mgmt import containerregistry
client = ContainerRegistryManagementClient(ws._auth,subscription_id)
result= client.registries.list_credentials(resource_group_name, reg_name, custom_headers=None, raw=False)
username = result.username
password = result.passwords[0].value

### Deployment file
This is the deployment.json file that you will use to deploy your model.  Please see the other sample notebooks on using this file to deploy the new model you created.

In [ ]:
file = open('./deployment-template.json')
contents = file.read()
contents = contents.replace('__MODULE_NAME', module_name)
contents = contents.replace('__REGISTRY_NAME', reg_name)
contents = contents.replace('__REGISTRY_USER_NAME', username)
contents = contents.replace('__REGISTRY_PASSWORD', password)
contents = contents.replace('__REGISTRY_IMAGE_LOCATION', image.image_location)
with open('./deployment.json', 'wt', encoding='utf-8') as output_file:
    output_file.write(contents)